# MAP5935 - Statistical Learning (Chapter 4 - Classification)

**Prof. Christian Jäkel**

https://www.statlearning.com/

## *The Stock Market Data*

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import subplots
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS, summarize)

C:\Users\lacor\anaconda3\envs\islp\Lib\site-packages\scipy\__init__.py:145: UserWarning: A NumPy version >=1.21.6 and <1.28.0 is required for this version of SciPy (detected version 2.0.1)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


ImportError: DLL load failed while importing _multiarray_umath: Não foi possível encontrar o módulo especificado.

ImportError: numpy.core.multiarray failed to import

In [2]:
from ISLP import confusion_table
from ISLP.models import contrast
from sklearn.discriminant_analysis import \
(LinearDiscriminantAnalysis as LDA ,
QuadraticDiscriminantAnalysis as QDA)
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

ImportError: DLL load failed while importing _multiarray_umath: Não foi possível encontrar o módulo especificado.

ImportError: numpy.core.multiarray failed to import

In [3]:
Smarket = load_data('Smarket')
Smarket

NameError: name 'load_data' is not defined

In [ ]:
Smarket.columns

In [ ]:
Smarket.info()

In [ ]:
corrvars = Smarket.columns.drop(['Today', 'Direction'])
Smarket[corrvars].corr()

In [ ]:
Smarket.plot(y='Volume')

## *Logistic Regression*

In [ ]:
allvars = Smarket.columns.drop(['Today', 'Direction', 'Year'])

# Model Specification
design = MS(allvars)

# Model Fit
X = design.fit_transform(Smarket)
y = Smarket.Direction == 'Up'
glm = sm.GLM(y, X, family=sm.families.Binomial())

# Model Results
results = glm.fit()
summarize(results)

- None of *lags* are statistically signiticant in explaining the `Direction`

In [ ]:
print('params>')
print(results.params)
print('\np-values>')
print(results.pvalues)

In [ ]:
# Probability that the market will go up, given values of the predictors
probs = results.predict()
probs [:10]

In [ ]:
labels = np.array(['Down']*1250)
labels[probs >0.5] = "Up"

In [ ]:
confusion_table(labels , Smarket.Direction)

In [ ]:
train = (Smarket.Year < 2005)
Smarket_train = Smarket.loc[train]
Smarket_test = Smarket.loc[~train]
Smarket_test.shape

In [ ]:
# Logistic regression model using only the subset of the observations that correspond to dates before 2005
X_train , X_test = X.loc[train], X.loc[~train]
y_train , y_test = y.loc[train], y.loc[~train]

glm_train = sm.GLM(y_train, X_train, family=sm.families.Binomial())

results = glm_train.fit()
probs = results.predict(exog=X_test)

In [ ]:
D = Smarket.Direction
L_train , L_test = D.loc[train], D.loc[~train]

In [ ]:
labels = np.array(['Down']*252)
labels[probs > 0.5] = 'Up'
confusion_table(labels , L_test)

In [ ]:
np.mean(labels == L_test), np.mean(labels != L_test)

In [ ]:
# Using variables more related with Direction
model = MS(['Lag1', 'Lag2']).fit(Smarket)
X = model.transform(Smarket)
X_train , X_test = X.loc[train], X.loc[~train]
glm_train = sm.GLM(y_train, X_train, family=sm.families.Binomial())
results = glm_train.fit()
probs = results.predict(exog=X_test)
labels = np.array(['Down']*252)
labels[probs >0.5] = 'Up'
confusion_table(labels , L_test)

In [ ]:
print('Overall Accuracy')
(35+106) /252 ,106/(106+76)

In [ ]:
newdata = pd.DataFrame({'Lag1':[1.2, 1.5], 'Lag2':[1.1, -0.8]})
newX = model.transform(newdata)
results.predict(newX)

## *Linear Disctiminant Analysis (LDA)*

In [ ]:
lda = LDA(store_covariance=True)

In [ ]:
X_train , X_test = [M.drop(columns=['intercept']) for M in [X_train , X_test]]
lda.fit(X_train , L_train)

In [ ]:
lda.means_

In [ ]:
lda.classes_

In [ ]:
lda.priors_

In [ ]:
lda.scalings_

In [ ]:
lda_pred = lda.predict(X_test)
confusion_table(lda_pred , L_test)

In [ ]:
lda_prob = lda.predict_proba(X_test)
np.all(np.where(lda_prob[:,1] >= 0.5, 'Up','Down') == lda_pred)

## *Quadratic Disctiminant Analysis (QDA)*

In [ ]:
qda = QDA(store_covariance=True)
qda.fit(X_train , L_train)

In [ ]:
qda.means_ , qda.priors_

In [ ]:
# First Class covariance
qda.covariance_ [0]

In [ ]:
qda_pred = qda.predict(X_test)
confusion_table(qda_pred , L_test)

In [ ]:
print(f'Percentage of cases predictect accurately = {np.mean(qda_pred == L_test)*100.0:.2f}%')

## *Naive Bayes*

In [ ]:
# Useful with no assuptioon Normal Distribution, small number of observations
NB = GaussianNB()
NB.fit(X_train , L_train)

In [ ]:
NB.classes_

In [ ]:
NB.class_prior_

In [ ]:
print(f'Theta=\n{NB.theta_}')
print(f'\nVariance=\n{NB.var_}')

In [ ]:
X_train[L_train == 'Down'].mean()

In [ ]:
X_train[L_train == 'Down'].var(ddof=0)

In [ ]:
nb_labels = NB.predict(X_test)
confusion_table(nb_labels , L_test)

Naive Bayes performs well on these data, with accurate predictions over 59% of the time. This is slightly worse than QDA, but much better than LDA.

In [ ]:
NB.predict_proba(X_test)[:5]

## *KNN - K-Nearest Neighbors*

In [ ]:
knn1 = KNeighborsClassifier(n_neighbors =1)
knn1.fit(X_train , L_train)
knn1_pred = knn1.predict(X_test)
confusion_table(knn1_pred , L_test)

In [ ]:
(83+43)/252, np.mean(knn1_pred == L_test)

In [ ]:
knn3 = KNeighborsClassifier(n_neighbors =3)
knn3_pred = knn3.fit(X_train , L_train).predict(X_test)
np.mean(knn3_pred == L_test)

In [ ]:
#import numpy as np
#import pandas as pd

#from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
# --- Assumindo que você já tem: X (Lag1, Lag2) e y = Direction (Up/Down) ---
# Ex.: X = df[['Lag1','Lag2']]
#      y = df['Direction']  # dtype category (Up/Down)

# 1) Split temporal (ex.: últimos 200 para teste)
#n_test = 200
#X_train, X_test = X.iloc[:-n_test], X.iloc[-n_test:]
#y_train, y_test = y.iloc[:-n_test], y.iloc[-n_test:]

# 2) Treinar KNN(3) e prever no teste
#knn3 = KNeighborsClassifier(n_neighbors=3)
y_pred = knn3.fit(X_train, y_train).predict(X_test)

# 3) Remontar DataFrame de previsões com índices originais
idx_test = y_test.index
pred_df = pd.DataFrame({
    'idx':   idx_test,
    'Lag1':  X_test['Lag1'].values,
    'Lag2':  X_test['Lag2'].values,
    'y_true': y_test.astype(str).values,
    'y_pred': pd.Series(y_pred).astype(str).values
}).set_index('idx')

# 4) Print rápido (primeiras linhas)
pred_view = pred_df.assign(
    correct = (pred_df['y_true'] == pred_df['y_pred'])
)
print(pred_view.head(10))   # remova .head(10) para ver tudo

# 5) Métricas
acc = np.mean(pred_df['y_true'] == pred_df['y_pred'])
print(f"Acurácia KNN(3) no teste: {acc:.3f}")

print("\nMatriz de confusão:")
print(confusion_matrix(pred_df['y_true'], pred_df['y_pred'], labels=['True','False']))

print("\nRelatório de classificação:")
print(classification_report(pred_df['y_true'], pred_df['y_pred'], target_names=['True','False']))

# 6) Plot da “evolução” de direções (Up/Down) ao longo do índice de teste
#    Como não temos preços, codificamos Up=+1, Down=-1 e mostramos acertos/erros.
# map_dir = {'Up': 1, 'up': 1, 'DOWN': -1, 'Down': -1, 'down': -1}
map_dir = {'True': 1, 'False': -1}
y_true_num = pd.Series(pred_df['y_true']).map(map_dir)
y_pred_num = pd.Series(pred_df['y_pred']).map(map_dir)

t = np.arange(len(pred_df))  # eixo x sequencial no período de teste
ok = (y_true_num == y_pred_num)

fig, ax = plt.subplots(figsize=(10, 3.5))
# verdade (linha fina)
ax.plot(t, y_true_num.values, linewidth=1, label='Direção real (+1=Up, -1=Down)')
# acertos (marcadores sólidos)
ax.scatter(t[ok & (y_pred_num==1)],  y_pred_num[ok & (y_pred_num==1)],  marker='^', s=40, label='Acerto Up')
ax.scatter(t[ok & (y_pred_num==-1)], y_pred_num[ok & (y_pred_num==-1)], marker='v', s=40, label='Acerto Down')
# erros (marcadores vazados)
ax.scatter(t[~ok & (y_pred_num==1)],  y_pred_num[~ok & (y_pred_num==1)],  marker='^', s=80, facecolors='none', label='Erro Up')
ax.scatter(t[~ok & (y_pred_num==-1)], y_pred_num[~ok & (y_pred_num==-1)], marker='v', s=80, facecolors='none', label='Erro Down')

ax.set_yticks([-1, 1])
ax.set_yticklabels(['Down (-1)', 'Up (+1)'])
ax.set_xlabel('Instante no conjunto de teste (ordem temporal)')
ax.set_title('Sequência de direções com previsões KNN(3)')
ax.legend(loc='best')
plt.tight_layout()
plt.show()


## *Conceptual Exercises*

### (6) Suppose we collect data for a group of students in a statistics class with variables $X1$ = hours studied, $X2$ = undergrad GPA, and $Y$ = receive an A. We fit a logistic regression and produce estimated coefficient, $\hat{\beta}_0=-6$, $\hat{\beta}_1=0.05$, $\hat{\beta}_2=1$.

### (a) Estimate the probability that a student who studies for $40$h and has an undergrad GPA of $3.5$ gets an A in the class.

**Solution**: We fitted a logistic regression model for the probability of receiving an **A**:

$$
\Pr(Y=1\mid X_1, X_2)= \frac{1}{1+\exp(-\eta)},\quad 
\eta=\hat\beta_0+\hat\beta_1 X_1+\hat\beta_2 X_2,
$$
with estimated coefficients $\hat\beta_0=-6$, $\hat\beta_1=0.05$ (hours studied $X_1$), and $\hat\beta_2=1$ (undergrad GPA $X_2$).


$$
\eta = -6 + 0.05(40) + 1(3.5) = -6 + 2 + 3.5 = -0.5.
$$
$$
\Pr(Y=1\mid X_1=40, X_2=3.5)=\frac{1}{1+\exp(0.5)}\approx 0.378.
$$

$\implies$ **Probability of getting an A = ~0.38**.

### (b) How many hours would the student in part (a) need to study to have a $50\%$ chance of getting an A in the class?

A probability of $0.5$ occurs when $\eta=0$. 

Now solving for $h$ (no. of study hours):

$$
0 = -6 + 0.05h + 1(3.5) \quad\Rightarrow\quad 0.05h = 2.5 \quad\Rightarrow\quad h=50.
$$

### (7) Suppose that we wish to predict whether a given stock will issue a dividend this year (“Yes” or “No”) based on X, last year’s percent profit. We examine a large number of companies and discover that the mean value of $X$ for companies that issued a dividend was $\bar{X} = 10$, while the mean for those that didn’t was $\bar{X} = 0$. In addition, the variance of $X$ for these two sets of companies was $\hat\sigma^2=36$. Finally, $80\%$ of companies issued dividends. Assuming that $X$ follows a normal distribution, predict the probability that a company will issue a dividend this year given that its percentage profit was $X = 4$ last year.

**Gaussian Naive Bayes with Bayes’ Theorem**

Let $Y\in\{\text{Yes},\text{No}\}$ indicate issuing a dividend.  

Given:
- Priors: $P(Y=\text{Yes})=0.8,\; P(Y=\text{No})=0.2$.
- Class-conditional $X\mid Y\sim \mathcal N(\mu_Y,\sigma^2)$ with common variance $\sigma^2=36$ ($\sigma=6$).
  - $\mu_{\text{Yes}}=10,\; \mu_{\text{No}}=0$.

For $x=4$, the normal density is
$$
f(x\mid\mu,\sigma^2)=\frac{1}{\sqrt{2\pi}\sigma}\exp\!\left(-\frac{(x-\mu)^2}{2\sigma^2}\right).
$$

Computing the likelihoods (the prefactor cancels since $\sigma$ is common):
$$
\begin{aligned}
f(4\mid Y=\text{Yes}) &\propto \exp\!\left(-\frac{(4-10)^2}{72}\right) = e^{-1/2},\\
f(4\mid Y=\text{No})  &\propto \exp\!\left(-\frac{(4-0)^2}{72}\right) = e^{-2/9}.
\end{aligned}
$$

From Bayes’ theorem:
$$
P(Y=\text{Yes}\mid X=4)=
\frac{f(4\mid \text{Yes})P(\text{Yes})}
     {f(4\mid \text{Yes})P(\text{Yes}) + f(4\mid \text{No})P(\text{No}) }.
$$

Numerically:
$$
\frac{0.8\,e^{-1/2}}{0.8\,e^{-1/2}+0.2\,e^{-2/9}}
=\frac{0.8\cdot 0.6065}{0.8\cdot 0.6065 + 0.2\cdot 0.8007}
\approx \frac{0.4852}{0.6454}
\approx 0.75.
$$

**The probability the company will issue a dividend given $X=4$ is ≈ 0.75**.


## *Applies Exercises*

### (13) This question should be answered using the `Weekly` data set, which is part of the ISLP package. This data is similar in nature to the Smarket data from this chapter’s lab, except that it contains $1,089$ weekly returns for $21$ years, from the beginning of $1990$ to the end of $2010$.

### (a) Produce some numerical and graphical summaries of the `Weekly` data. Do there appear to be any patterns?

In [ ]:
Weekly = load_data('Weekly')
Weekly.info()

In [ ]:
# ---- setup ----
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.set_context("notebook")
pd.set_option("display.float_format", lambda x: f"{x:,.4f}")

# df = Weekly   # <-- make sure your DataFrame is named `Weekly`
df = Weekly.copy()

# ---- 1) Numerical summaries ----
display(df.describe(include="all").T)

# Class balance
vc = df["Direction"].value_counts()
vc_norm = df["Direction"].value_counts(normalize=True).rename("proportion")
display(pd.concat([vc.rename("count"), vc_norm], axis=1))

# Correlations among numeric columns
corr = df.select_dtypes(include=[np.number]).corr()
display(corr.style.background_gradient(cmap="coolwarm", vmin=-1, vmax=1))

# ---- 2) Graphical summaries ----
fig, axes = plt.subplots(2, 2, figsize=(12, 9))

# (a) Returns today
axes[0,0].hist(df["Today"], bins=40, edgecolor="k")
axes[0,0].set_title("Histogram of Today (weekly return)")
axes[0,0].set_xlabel("Return"); axes[0,0].set_ylabel("Count")

# (b) Volume over time
df.groupby("Year")["Volume"].mean().plot(ax=axes[0,1])
axes[0,1].set_title("Average Weekly Volume by Year")
axes[0,1].set_xlabel("Year"); axes[0,1].set_ylabel("Avg Volume")

# (c) Today vs Lag1
axes[1,0].scatter(df["Lag1"], df["Today"], s=12, alpha=0.6)
axes[1,0].axhline(0, ls="--", lw=1, c="grey"); axes[1,0].axvline(0, ls="--", lw=1, c="grey")
axes[1,0].set_title("Today vs Lag1"); axes[1,0].set_xlabel("Lag1"); axes[1,0].set_ylabel("Today")

# (d) Pairwise relationships (small sample for readability)
sns.pairplot(
    df.sample(min(400, len(df)), random_state=1),
    vars=["Lag1","Lag2","Lag3","Lag4","Lag5","Today"],
    corner=True, plot_kws=dict(s=12, alpha=0.5)
)
plt.tight_layout()
plt.show()


What patterns usually appear in this dataset

- **Returns center near zero**. The weekly returns (`Today` and the lags) are roughly symmetric and concentrated around 0, with occasional outliers.

- **Slight class imbalance**. `Direction` typically has slightly more *Up* than *Down* weeks (but not dramatically so).

- **Weak linear signal from lags**. Correlations between Today and `Lag1`–`Lag5` are very small (often close to 0), suggesting limited linear predictive power from past week returns.

- **Clear time trend in trading activity**. Volume increases over the years (plot of average `volume` by Year shows an upward trend), while returns do not show a comparable trend.

- **No obvious low-dimensional structure**. Scatterplots among Lag variables and Today look diffuse, indicating weak simple relationships.

> These observations motivate trying classification models but set expectations that simple linear relationships using only lagged returns may have limited predictive strength.

### (b) Use the full data set to perform a logistic regression with `Direction` as the response and the five lag variables plus `Volume` as predictors. Use the summary function to print the results. Do any of the predictors appear to be statistically significant? If so, which ones?

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Response as binary: 1 = "Up", 0 = "Down"
df["Up"] = (df["Direction"].astype(str) == "Up").astype(int)

# Fit logistic regression with all five lags + Volume
model = smf.logit("Up ~ Lag1 + Lag2 + Lag3 + Lag4 + Lag5 + Volume", data=df)
res = model.fit(disp=False)

# Full regression table
print(res.summary())

# Identify statistically significant predictors at alpha=0.05
alpha = 0.05
sig = res.pvalues[res.pvalues < alpha].drop("Intercept", errors="ignore")
print("\nSignificant at α = 0.05:")
print(sig.sort_values())

# (Optional) Odds ratios with 95% CIs
params = res.params
conf = res.conf_int()
or_table = pd.DataFrame({
    "OR": np.exp(params),
    "2.5%": np.exp(conf[0]),
    "97.5%": np.exp(conf[1]),
    "p-value": res.pvalues
})
display(or_table)


**Interpretation** (expected with the Weekly data): only `Lag2` typically appears statistically significant at the 5% level (positive coefficient), while `Lag1`, `Lag3`, `Lag4`, `Lag5`, and `Volume` are not statistically significant.

### (c) Compute the confusion matrix and overall fraction of correct predictions. Explain what the confusion matrix is telling you about the types of mistakes made by logistic regression.

In [ ]:
def calc_acc (cm):
    acc = (y_pred == y_test).mean()
    tp = cm.loc["Up","Up"]; tn = cm.loc["Down","Down"]
    fp = cm.loc["Up","Down"]; fn = cm.loc["Down","Up"]
    tpr = tp / (tp + fn) if (tp + fn) else np.nan  # sensitivity for 'Up'
    tnr = tn / (tn + fp) if (tn + fp) else np.nan  # specificity for 'Down'

    print(f"Overall test accuracy (2009–2010): {acc:.3f}")
    print(f"Sensitivity (Up): {tpr:.3f} | Specificity (Down): {tnr:.3f}")

In [ ]:
# --- fit (again, for a self-contained cell) ---
df["Up"] = (df["Direction"].astype(str) == "Up").astype(int)

logit = smf.logit("Up ~ Lag1 + Lag2 + Lag3 + Lag4 + Lag5 + Volume", data=df).fit(disp=False)

# --- in-sample predictions, default 0.5 threshold ---
probs = logit.predict(df)
pred = pd.Series(np.where(probs >= 0.5, "Up", "Down"), index=df.index, name="Predicted")
actual = df["Direction"].astype(str)

# confusion matrix with fixed order
labels = ["Down", "Up"]
cm_log = pd.crosstab(pred, actual).reindex(index=labels, columns=labels, fill_value=0)
display(cm_log)

# accuracy and a couple of helpful rates
calc_acc (cm_log)

### (d) Now fit the logistic regression model using a training data period from $1990$ to $2008$, with `Lag2` as the only predictor. Compute the confusion matrix and the overall fraction of correct predictions for the held out data (that is, the data from $2009$ and $2010$).

In [ ]:
# Train: 1990–2008 | Test: 2009–2010
train_idx = df["Year"] <= 2008
test_idx  = df["Year"] >= 2009

# Fit logistic regression with Lag2 only (on training period)
logit_lag2 = smf.logit("Up ~ Lag2", data=df.loc[train_idx]).fit(disp=False)

# Predict on held-out 2009–2010
probs_test = logit_lag2.predict(df.loc[test_idx])
pred_test  = np.where(probs_test >= 0.5, "Up", "Down")

actual_test = df.loc[test_idx, "Direction"].astype(str)

# Confusion matrix (rows = predicted, cols = actual)
labels = ["Down", "Up"]
cm_log_l2 = pd.crosstab(
    pd.Series(pred_test, index=actual_test.index, name="Predicted"),
    actual_test,
).reindex(index=labels, columns=labels, fill_value=0)
display(cm_log_l2)

# Overall accuracy (fraction correct)
calc_acc (cm_log_l2)


### (e) Repeat (d) using LDA.

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# Data
df["Direction"] = df["Direction"].astype(str)

# Train/Test split
train_idx = df["Year"] <= 2008
test_idx  = df["Year"] >= 2009

X_train = df.loc[train_idx, ["Lag2"]].to_numpy()
y_train = df.loc[train_idx, "Direction"].to_numpy()

X_test  = df.loc[test_idx, ["Lag2"]].to_numpy()
y_test  = df.loc[test_idx, "Direction"].to_numpy()

# Fit LDA
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)

# Predict on test period (2009–2010)
y_pred = lda.predict(X_test)

# Confusion matrix (rows = predicted, cols = actual)
labels = ["Down", "Up"]
cm_lda = pd.crosstab(
    pd.Series(y_pred, index=df.index[test_idx], name="Predicted"),
    pd.Series(y_test, index=df.index[test_idx], name="Actual")
).reindex(index=labels, columns=labels, fill_value=0)
display(cm_lda)

# Overall accuracy and a couple of rates
calc_acc (cm_lda)


### (f) Repeat (d) using QDA.

In [ ]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

# Train/Test split
train_idx = df["Year"] <= 2008
test_idx  = df["Year"] >= 2009

X_train = df.loc[train_idx, ["Lag2"]].to_numpy()
y_train = df.loc[train_idx, "Direction"].to_numpy()

X_test  = df.loc[test_idx, ["Lag2"]].to_numpy()
y_test  = df.loc[test_idx, "Direction"].to_numpy()

# Fit QDA
qda = QuadraticDiscriminantAnalysis()
qda.fit(X_train, y_train)

# Predict on test set (2009–2010)
y_pred = qda.predict(X_test)

# Confusion matrix (rows = predicted, cols = actual)
labels = ["Down", "Up"]
cm_qda = pd.crosstab(
    pd.Series(y_pred, index=df.index[test_idx], name="Predicted"),
    pd.Series(y_test, index=df.index[test_idx], name="Actual")
).reindex(index=labels, columns=labels, fill_value=0)
display(cm_qda)

# Overall accuracy and helpful rates
calc_acc (cm_qda)


### Repeat (g) using KNN with K = 1.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Train/Test split
train_idx = df["Year"] <= 2008
test_idx  = df["Year"] >= 2009

X_train = df.loc[train_idx, ["Lag2"]].to_numpy()
y_train = df.loc[train_idx, "Direction"].to_numpy()

X_test  = df.loc[test_idx, ["Lag2"]].to_numpy()
y_test  = df.loc[test_idx, "Direction"].to_numpy()

# Fit KNN with K=1
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)

# Predict on test period (2009–2010)
y_pred = knn.predict(X_test)

# Confusion matrix (rows = predicted, cols = actual)
labels = ["Down", "Up"]
cm_knn = pd.crosstab(
    pd.Series(y_pred, index=df.index[test_idx], name="Predicted"),
    pd.Series(y_test, index=df.index[test_idx], name="Actual")
).reindex(index=labels, columns=labels, fill_value=0)
display(cm_knn)

# Overall accuracy and helpful rates
calc_acc (cm_knn)


### (h) Repeat (d) using naive Bayes.

In [ ]:
from sklearn.naive_bayes import GaussianNB

# Train/Test split
train_idx = df["Year"] <= 2008
test_idx  = df["Year"] >= 2009

X_train = df.loc[train_idx, ["Lag2"]].to_numpy()
y_train = df.loc[train_idx, "Direction"].to_numpy()

X_test  = df.loc[test_idx, ["Lag2"]].to_numpy()
y_test  = df.loc[test_idx, "Direction"].to_numpy()

# Fit Gaussian Naive Bayes
gnb = GaussianNB()
gnb.fit(X_train, y_train)

# Predict on test (2009–2010)
y_pred = gnb.predict(X_test)

# Confusion matrix (rows = predicted, cols = actual)
labels = ["Down", "Up"]
cm_nb = pd.crosstab(
    pd.Series(y_pred, index=df.index[test_idx], name="Predicted"),
    pd.Series(y_test, index=df.index[test_idx], name="Actual")
).reindex(index=labels, columns=labels, fill_value=0)
display(cm_nb)

# Overall accuracy + a couple of helpful rates
calc_acc (cm_nb)


### (i) Which of these methods appears to provide the best results on this data?

In [ ]:
print("\n==> logit\n")
calc_acc(cm_log)
print("\n==> logit + lag2\n")
calc_acc(cm_log_l2)
print("\n==> lda\n")
calc_acc(cm_lda)
print("\n==> qda\n")
calc_acc(cm_qda)
print("\n==> knn, k=1\n")
calc_acc(cm_knn)
print("\n==> Naive Bayes\n")
calc_acc(cm_nb)

**Short answer:** By plain accuracy, **all models tie at 0.587**, which equals the “always predict Up” baseline — so none truly “wins” on accuracy.

**A better tie-breaker:** use **balanced accuracy** = (Sensitivity + Specificity)/2  
(combats the skew toward predicting *Up*).

| Model               | Sensitivity | Specificity | Balanced Acc. |
|---------------------|-------------|-------------|----------------|
| Logit (all vars)    | 0.921       | 0.112       | **0.517**      |
| Logit (Lag2 only)   | 0.918       | 0.209       | **0.564** 🔹   |
| **LDA (Lag2 only)** | 0.918       | 0.209       | **0.564** 🔹   |
| QDA (Lag2 only)     | 1.000       | 0.000       | 0.500          |
| KNN, K=1 (Lag2)     | 0.492       | 0.512       | 0.502          |
| Naive Bayes (Lag2)  | 1.000       | 0.000       | 0.500          |

**Verdict:** **LDA (Lag2)** and **Logit (Lag2)** are the best of this set (tied), offering the highest balanced accuracy and a modest boost in specificity while keeping high sensitivity.  
**Note:** Since overall accuracy matches the majority-class rate, these models have limited practical predictiveness on this test; consider threshold tuning, adding features, or alternative models (e.g., trees/ensembles) and time-series CV.


### (j) Experiment with different combinations of predictors, including possible transformations and interactions, for each of the methods. Report the variables, method, and associated confusion matrix that appears to provide the best results on the held out data. Note that you should also experiment with values for K in the KNN classifier.

> Train on **1990–2008**, test on **2009–2010**.  
> We try multiple predictor sets, simple transforms/interactions, and a K grid for KNN.  
> We report per-method winners and the **overall best** by **balanced accuracy** $(\text{TPR}+\text{TNR})/2$, along with the confusion matrix.



In [ ]:
import numpy as np
import pandas as pd

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

import statsmodels.api as sm


In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix

# ---------------------------
# 1) Data prep
# ---------------------------
df = Weekly.copy()
df = df.assign(
    Direction=df["Direction"].astype(str),
    Up=(df["Direction"].astype(str)=="Up").astype(int),
    Lag2_sq=df["Lag2"]**2,
    Lag1xLag2=df["Lag1"]*df["Lag2"],
    logV=np.log(df["Volume"].clip(lower=1e-12)),
    Lag2_pos=(df["Lag2"]>0).astype(int)
)

train_idx = df["Year"] <= 2008
test_idx  = df["Year"] >= 2009

y_train = df.loc[train_idx, "Direction"].to_numpy()
y_test  = df.loc[test_idx,  "Direction"].to_numpy()
y_train_bin = (y_train == "Up").astype(int)
y_test_bin  = (y_test  == "Up").astype(int)

# Candidate feature sets
feature_sets = {
    "Lag2"              : ["Lag2"],
    "Lag2 + Lag1"       : ["Lag2","Lag1"],
    "Lag2 + Lag2^2"     : ["Lag2","Lag2_sq"],
    "Lag2 + Lag1:Lag2"  : ["Lag2","Lag1","Lag1xLag2"],
    "Lags1-5"           : ["Lag1","Lag2","Lag3","Lag4","Lag5"],
    "Lags1-5 + logV"    : ["Lag1","Lag2","Lag3","Lag4","Lag5","logV"],
    "Lag2 + logV"       : ["Lag2","logV"],
    "Lag2_pos (indicator)": ["Lag2_pos"],        # step function on Lag2
}

# Helper to compute metrics from a confusion matrix
def summarize(cm, labels=("Down","Up")):
    # cm rows = pred, cols = actual, order by labels
    tn = cm[0,0]; fp = cm[0,1]
    fn = cm[1,0]; tp = cm[1,1]
    total = cm.sum()
    acc = (tp + tn) / total if total else np.nan
    tpr = tp / (tp + fn) if (tp + fn) else np.nan   # sensitivity for Up
    tnr = tn / (tn + fp) if (tn + fp) else np.nan   # specificity for Down
    bal = 0.5*(tpr + tnr) if np.isfinite(tpr) and np.isfinite(tnr) else np.nan
    return dict(acc=acc, tpr=tpr, tnr=tnr, bal=bal, tp=tp, tn=tn, fp=fp, fn=fn)

def confmat_df(cm, labels=("Down","Up")):
    return pd.DataFrame(cm, index=pd.Index(labels, name="Predicted"), columns=pd.Index(labels, name="Actual"))

# Containers for best per method
best = {}   # method -> dict with keys: name, Xcols, cm, metrics, extra (e.g., K)
overall_best = None

# ---------------------------
# 2) LOGISTIC REGRESSION (GLM Binomial with logit link)
# ---------------------------
for name, cols in feature_sets.items():
    X_train = df.loc[train_idx, cols]
    X_test  = df.loc[test_idx,  cols]

    X_train_sm = sm.add_constant(X_train)
    X_test_sm  = sm.add_constant(X_test)

    # Fit GLM Binomial (more stable than Logit for edge cases)
    glm = sm.GLM(y_train_bin, X_train_sm, family=sm.families.Binomial())
    res = glm.fit()

    # Predict with default 0.5 threshold
    p_test = res.predict(X_test_sm)
    y_pred = np.where(p_test >= 0.5, "Up", "Down")

    cm = confusion_matrix(y_test, y_pred, labels=["Down","Up"])
    metrics = summarize(cm)
    rec = dict(method="Logit", name=name, Xcols=cols, cm=cm, metrics=metrics)

    if "Logit" not in best or metrics["bal"] > best["Logit"]["metrics"]["bal"]:
        best["Logit"] = rec

# ---------------------------
# 3) LDA
# ---------------------------
for name, cols in feature_sets.items():
    X_train = df.loc[train_idx, cols].to_numpy()
    X_test  = df.loc[test_idx,  cols].to_numpy()
    lda = LinearDiscriminantAnalysis()
    lda.fit(X_train, y_train)
    y_pred = lda.predict(X_test)

    cm = confusion_matrix(y_test, y_pred, labels=["Down","Up"])
    metrics = summarize(cm)
    rec = dict(method="LDA", name=name, Xcols=cols, cm=cm, metrics=metrics)

    if "LDA" not in best or metrics["bal"] > best["LDA"]["metrics"]["bal"]:
        best["LDA"] = rec

# ---------------------------
# 4) QDA
# ---------------------------
for name, cols in feature_sets.items():
    X_train = df.loc[train_idx, cols].to_numpy()
    X_test  = df.loc[test_idx,  cols].to_numpy()
    qda = QuadraticDiscriminantAnalysis()
    # QDA can fail if a class has 0 variance along a feature; guard with try
    try:
        qda.fit(X_train, y_train)
        y_pred = qda.predict(X_test)
        cm = confusion_matrix(y_test, y_pred, labels=["Down","Up"])
        metrics = summarize(cm)
        rec = dict(method="QDA", name=name, Xcols=cols, cm=cm, metrics=metrics)
        if "QDA" not in best or metrics["bal"] > best["QDA"]["metrics"]["bal"]:
            best["QDA"] = rec
    except Exception as e:
        pass  # skip problematic specs

# ---------------------------
# 5) Gaussian Naive Bayes
# ---------------------------
for name, cols in feature_sets.items():
    X_train = df.loc[train_idx, cols].to_numpy()
    X_test  = df.loc[test_idx,  cols].to_numpy()
    gnb = GaussianNB()
    gnb.fit(X_train, y_train)
    y_pred = gnb.predict(X_test)

    cm = confusion_matrix(y_test, y_pred, labels=["Down","Up"])
    metrics = summarize(cm)
    rec = dict(method="NaiveBayes", name=name, Xcols=cols, cm=cm, metrics=metrics)

    if "NaiveBayes" not in best or metrics["bal"] > best["NaiveBayes"]["metrics"]["bal"]:
        best["NaiveBayes"] = rec

# ---------------------------
# 6) KNN (with scaling) — search K
# ---------------------------
K_grid = [1, 3, 5, 7, 9, 11, 15, 21, 31]

for name, cols in feature_sets.items():
    X_train = df.loc[train_idx, cols].to_numpy()
    X_test  = df.loc[test_idx,  cols].to_numpy()

    for K in K_grid:
        knn = make_pipeline(StandardScaler(with_mean=True, with_std=True), KNeighborsClassifier(n_neighbors=K))
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)

        cm = confusion_matrix(y_test, y_pred, labels=["Down","Up"])
        metrics = summarize(cm)
        rec = dict(method="KNN", name=f"{name} | K={K}", Xcols=cols, cm=cm, metrics=metrics, K=K)

        if "KNN" not in best or metrics["bal"] > best["KNN"]["metrics"]["bal"]:
            best["KNN"] = rec

# ---------------------------
# 7) Display per-method best and pick overall best
# ---------------------------
rows = []
for method, rec in best.items():
    m = rec["metrics"]
    rows.append([
        method,
        rec["name"],
        f"{m['acc']:.3f}",
        f"{m['tpr']:.3f}",
        f"{m['tnr']:.3f}",
        f"{m['bal']:.3f}"
    ])
    if (overall_best is None) or (m["bal"] > overall_best["metrics"]["bal"]):
        overall_best = rec

summary_df = pd.DataFrame(rows, columns=["Method","Best feature set / K","Accuracy","Sensitivity (Up)","Specificity (Down)","Balanced Acc."])
display(summary_df.sort_values("Balanced Acc.", ascending=False, key=lambda s: s.astype(float)))

print("\n🏆 Best overall by Balanced Accuracy:")
m = overall_best["metrics"]
print(f"- Method: {overall_best['method']}")
print(f"- Features: {overall_best['name']} -> {overall_best['Xcols']}")
print(f"- Accuracy: {m['acc']:.3f} | Sensitivity (Up): {m['tpr']:.3f} | Specificity (Down): {m['tnr']:.3f} | Balanced Acc.: {m['bal']:.3f}")

print("\nConfusion Matrix (rows=Pred, cols=Actual):")
display(confmat_df(overall_best["cm"]))
